In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.style.use("seaborn-darkgrid") # style for matplotlib

In [5]:
from tqdm import tqdm # loading bar

In [6]:
path="../input/face-recognition-30/dataset/"
files=os.listdir(path)
files

['Courteney_Cox',
 'arnold_schwarzenegger',
 'bhuvan_bam',
 'hardik_pandya',
 'David_Schwimmer',
 'Matt_LeBlanc',
 'Simon_Helberg',
 'scarlett_johansson',
 'Pankaj_Tripathi',
 'Matthew_Perry',
 'sylvester_stallone',
 'messi',
 'Jim_Parsons',
 'random_person',
 'Lisa_Kudrow',
 'mohamed_ali',
 'brad_pitt',
 'ronaldo',
 'virat_kohli',
 'angelina_jolie',
 'Kunal_Nayya',
 'manoj_bajpayee',
 'Sachin_Tendulka',
 'Jennifer_Aniston',
 'dhoni',
 'pewdiepie',
 'aishwarya_rai',
 'Johnny_Galeck',
 'ROHIT_SHARMA',
 'suresh_raina']

In [7]:
len(files)

30

In [8]:
image_array=[] # a list that will converted to array
label_array=[] # label of file as number

# loop through each sub-folder in train
for i in range(len(files)):
    file_sub=os.listdir(path+files[i])
    
    # loop trough files in sub-folder
    for k in tqdm(range(len(file_sub))):
        try:
            img=cv2.imread(path+files[i]+"/"+file_sub[k])
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img=cv2.resize(img,(96,96))
            image_array.append(img)
            label_array.append(i)
        except:
            pass

100%|██████████| 319/319 [00:01<00:00, 183.25it/s]


In [9]:
# look what we have got
image_array[0]

array([[[39, 28, 21],
        [34, 25, 18],
        [44, 37, 28],
        ...,
        [ 8,  0,  1],
        [ 6,  0,  0],
        [ 8,  1,  0]],

       [[42, 29, 21],
        [32, 22, 13],
        [41, 32, 23],
        ...,
        [ 9,  0,  1],
        [ 7,  1,  1],
        [ 7,  2,  0]],

       [[34, 21, 13],
        [40, 30, 21],
        [53, 45, 34],
        ...,
        [ 9,  0,  1],
        [ 6,  0,  0],
        [ 5,  0,  0]],

       ...,

       [[11,  4,  1],
        [16,  7,  3],
        [25, 16,  9],
        ...,
        [28, 21, 15],
        [37, 29, 26],
        [37, 27, 25]],

       [[17,  6,  4],
        [25, 14, 10],
        [26, 17, 10],
        ...,
        [27, 19, 16],
        [30, 22, 19],
        [37, 27, 25]],

       [[18,  9,  4],
        [31, 22, 15],
        [26, 17, 10],
        ...,
        [27, 17, 14],
        [30, 20, 18],
        [40, 31, 28]]], dtype=uint8)

In [10]:
# look what we have got
label_array[0:5]

[0, 0, 0, 0, 0]

In [11]:
import gc
# free up memory space
gc.collect()

121

In [12]:
# convert both of them to array
# divide by 255 to scale image from 0-255 to 0-1
image_array=np.array(image_array)/255.0
label_array=np.array(label_array)

In [13]:
image_array

array([[[[0.15294118, 0.10980392, 0.08235294],
         [0.13333333, 0.09803922, 0.07058824],
         [0.17254902, 0.14509804, 0.10980392],
         ...,
         [0.03137255, 0.        , 0.00392157],
         [0.02352941, 0.        , 0.        ],
         [0.03137255, 0.00392157, 0.        ]],

        [[0.16470588, 0.11372549, 0.08235294],
         [0.1254902 , 0.08627451, 0.05098039],
         [0.16078431, 0.1254902 , 0.09019608],
         ...,
         [0.03529412, 0.        , 0.00392157],
         [0.02745098, 0.00392157, 0.00392157],
         [0.02745098, 0.00784314, 0.        ]],

        [[0.13333333, 0.08235294, 0.05098039],
         [0.15686275, 0.11764706, 0.08235294],
         [0.20784314, 0.17647059, 0.13333333],
         ...,
         [0.03529412, 0.        , 0.00392157],
         [0.02352941, 0.        , 0.        ],
         [0.01960784, 0.        , 0.        ]],

        ...,

        [[0.04313725, 0.01568627, 0.00392157],
         [0.0627451 , 0.02745098, 0.01176471]

In [14]:
from sklearn.model_selection import train_test_split
# split datatest to : train (85%) and test (15%)
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)

In [15]:
# create the model using classification
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential,Model,load_model

In [16]:
model=Sequential()
# use MobileNetV2 as an pretrained model 
pretrained_model=tf.keras.applications.EfficientNetB0(input_shape=(96,96,3),include_top=False,weights="imagenet")
model.add(pretrained_model)
model.add(layers.GlobalAveragePooling2D())
# add dropout to increase accuracy by not overfitting
model.add(layers.Dropout(0.3))
# add dense layer as final output
model.add(layers.Dense(1))
model.summary()

16705208/16705208 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 3, 3, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 4,050,852
Trainable params: 4,008,829
Non-trainable params: 42,023
_________________________________________________________________


In [17]:
# compile the model
model.compile(optimizer="adam",loss="mean_squared_error",metrics=["mae"])

In [18]:
# creating a chechpoint to save model at best accuarcy

ckp_path="trained_model/model"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=ckp_path,
                                                   monitor="val_mae",
                                                   mode="auto",
                                                   save_best_only=True,
                                                   save_weights_only=True)

In [19]:
# create a lr reducer which decrease learning rate when accuarcy does not increase
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(factor=0.9,monitor="val_mae",
                                             mode="auto",cooldown=0,
                                             patience=5,verbose=1,min_lr=1e-6)
# patience : wait till 5 epoch
# verbose : show accuracy every 1 epoch
# min_lr=minimum learning rate

In [20]:
EPOCHS=250
BATCH_SIZE=64

history=model.fit(X_train,
                 Y_train,
                 validation_data=(X_test,Y_test),
                 batch_size=BATCH_SIZE,
                 epochs=EPOCHS,
                 callbacks=[model_checkpoint,reduce_lr]
                 )


Epoch 1/250


2023-04-15 05:04:21.666208: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


212/212 [==============================] - 75s 132ms/step - loss: 48.6125 - mae: 5.2234 - val_loss: 130.9044 - val_mae: 9.3121 - lr: 0.0010
Epoch 2/250
212/212 [==============================] - 23s 110ms/step - loss: 19.5191 - mae: 3.1901 - val_loss: 84.5668 - val_mae: 7.4695 - lr: 0.0010
Epoch 3/250
212/212 [==============================] - 23s 110ms/step - loss: 13.0086 - mae: 2.5075 - val_loss: 76.5414 - val_mae: 7.1062 - lr: 0.0010
Epoch 4/250
212/212 [==============================] - 23s 110ms/step - loss: 9.9551 - mae: 2.1557 - val_loss: 43.4295 - val_mae: 5.1040 - lr: 0.0010
Epoch 5/250
212/212 [==============================] - 23s 108ms/step - loss: 8.1676 - mae: 1.9132 - val_loss: 77.4033 - val_mae: 7.1351 - lr: 0.0010
Epoch 6/250
212/212 [==============================] - 23s 106ms/step - loss: 7.0835 - mae: 1.7596 - val_loss: 68.3748 - val_mae: 6.6791 - lr: 0.0010
Epoch 7/250
212/212 [==============================] - 23s 107ms/step - loss: 5.1124 - mae: 1.4896 - val_los

In [21]:
model.load_weights(ckp_path)

In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [23]:
prediction_val=model.predict(X_test,batch_size=BATCH_SIZE)

38/38 [==============================] - 3s 27ms/step


In [24]:
prediction_val[:20]

array([[1.29363089e+01],
       [5.00449467e+00],
       [4.05181313e+00],
       [1.39345264e+01],
       [1.42125034e+01],
       [2.65003276e+00],
       [1.20614977e+01],
       [1.80131721e+01],
       [1.97481995e+01],
       [1.58019028e+01],
       [5.60304356e+00],
       [1.18319016e+01],
       [8.91456413e+00],
       [1.34355850e+01],
       [2.29922943e+01],
       [1.31233015e+01],
       [1.80397339e+01],
       [2.06177101e+01],
       [1.86725855e-02],
       [2.56661568e+01]], dtype=float32)